# Задания

Применим линейную регрессию на практике – попробуем предсказать стоимость машин и понять, от каких факторов зависит ценообразование на автомобили. Помимо этого узнаем, какие переменные важны для прогнозирования и насколько хорошо полученная модель описывает данные. 

Задание:
1. Загрузите данные, проверьте правильность, наличие пропущенных значений, типы данных.
2. Создайте новый признак – марку автомобиля (company). Машины каких производителей встречаются в датасете? Далее исправьте названия и проверьте изменения.
3. Оставьте только часть предикторов, после чего посчитайте корреляцию между price и другими переменными.
4. Преобразуйте категориальные переменные с помощью pd.get_dummies(). 
5. Постройте модель с одним предиктором цены – horsepower. Какой процент изменчивости объясняет полученная модель? (\(R^2\))
6. Далее – две модели (со всеми предикторами и со всеми, кроме марок машин). Обратите внимание на изменения в \(R^2\), коэффициентах и их значимости. Какую модель лучше оставить?

# Задача 1

Загрузите данные, проверьте правильность, наличие пропущенных значений, типы данных.

In [14]:
#импортируем необоходимые библиотеки
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [15]:
#загружаем данные
df = pd.read_csv('cars.csv')

In [16]:
#проверяем форматы значений, пропущенные ячейки
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

# Задача 2

Создайте новый признак – марку автомобиля (company). Машины каких производителей встречаются в датасете? Далее исправьте названия и проверьте изменения.

In [17]:
#добавили новый столбец, который из названия модели отбирает компанию. Столбец вывели в левый край
df.insert(0, 'company', df.CarName.str.split(' ').str[0])

In [18]:
#удалили ненужные стобцы
df = df.drop(['CarName', 'car_ID'], axis=1)

In [19]:
#посчитали количество уникальных компаний
df.company.nunique()

28

In [20]:
df.head()

,company,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,alfa-romero,3,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,alfa-romero,3,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,alfa-romero,1,gas,std,two,hatchback,rwd,front,94.5,171.2,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,audi,2,gas,std,four,sedan,fwd,front,99.8,176.6,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,audi,2,gas,std,four,sedan,4wd,front,99.4,176.6,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [21]:
#привели все значения к нижнему регистру и заменили названия компаний на правильные, чтобы убрать дубли
df['company'] = df.company.str.lower()
df['company'] = df.company.replace(['maxda', 'porcshce', 'toyouta', 'vokswagen', 'vw'], ['mazda', 'porsche', 'toyota', 'volkswagen', 'volkswagen'])

In [22]:
#заново посчитали количество уникальных компаний
df.company.nunique()

22

In [23]:
df.head()

,company,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,alfa-romero,3,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,alfa-romero,3,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,alfa-romero,1,gas,std,two,hatchback,rwd,front,94.5,171.2,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,audi,2,gas,std,four,sedan,fwd,front,99.8,176.6,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,audi,2,gas,std,four,sedan,4wd,front,99.4,176.6,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


# Задача 3

Отлично! Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:

'company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower'
также не забыв про то, что мы предсказываем – 'price'. 

После этого посчитайте корреляцию между price и другими переменными. Чему равна корреляция между price и horsepower? Ответ округлите до 2 знаков после точки.

In [24]:
#удалили ненужные столбцы
df = df.drop(['symboling', 'doornumber', 'enginelocation', 'carheight','fuelsystem', 'stroke', 'compressionratio', 'peakrpm', 'citympg', 'highwaympg'], axis=1)

In [25]:
#посчитали корреляцию между price и другими переменными
df.corr().round(2)

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price
wheelbase,1.00,0.87,0.80,0.78,0.57,0.49,0.35,0.58
carlength,0.87,1.00,0.84,0.88,0.68,0.61,0.55,0.68
carwidth,0.80,0.84,1.00,0.87,0.74,0.56,0.64,0.76
curbweight,0.78,0.88,0.87,1.00,0.85,0.65,0.75,0.84
enginesize,0.57,0.68,0.74,0.85,1.00,0.58,0.81,0.87
boreratio,0.49,0.61,0.56,0.65,0.58,1.00,0.57,0.55
horsepower,0.35,0.55,0.64,0.75,0.81,0.57,1.00,0.81
price,0.58,0.68,0.76,0.84,0.87,0.55,0.81,1.00


In [26]:
df.shape[1]

15

# Задача 4

Преобразуйте категориальные переменные с помощью pd.get_dummies(). 

Последний шаг в подготовке данных: линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), поэтому давайте применим pd.get_dummies(). Пример использования:

df_dummy = pd.get_dummies(data=cars[[список_столбцов_типа_object]], drop_first = True)
Не забудьте присоединить к полученному датасету столбцы с переменными других типов :) Также обратите внимание: мы должны использовать тот же набор колонок, который был на предыдущем шаге!

Сколько колонок теперь имеется в датасете?

In [27]:
#вывели таблицу с типами данных в стоблцах
x = df.dtypes.to_frame().reset_index().rename(columns={'index':'name_columns', 0:'type'})
x

,name_columns,type
0,company,object
1,fueltype,object
2,aspiration,object
3,carbody,object
4,drivewheel,object
5,wheelbase,float64
6,carlength,float64
7,carwidth,float64
8,curbweight,int64
9,enginetype,object


In [28]:
#Преобразовали категориальные переменные в фиктивные/индикаторные переменные.
df_dummy = pd.get_dummies(data=df[['company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 
                                   'enginetype', 'cylindernumber']], drop_first = True)

In [29]:
#из исходного дф удалили столбцы с типом object, чтобы потом добавить столбцы из предыдущего шага
df = df.drop(['company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 
                                   'enginetype', 'cylindernumber'], 1)

In [30]:
#объединили таблицы
df = pd.concat([df, df_dummy], axis=1)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 49 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   wheelbase              205 non-null    float64
 1   carlength              205 non-null    float64
 2   carwidth               205 non-null    float64
 3   curbweight             205 non-null    int64  
 4   enginesize             205 non-null    int64  
 5   boreratio              205 non-null    float64
 6   horsepower             205 non-null    int64  
 7   price                  205 non-null    float64
 8   company_audi           205 non-null    uint8  
 9   company_bmw            205 non-null    uint8  
 10  company_buick          205 non-null    uint8  
 11  company_chevrolet      205 non-null    uint8  
 12  company_dodge          205 non-null    uint8  
 13  company_honda          205 non-null    uint8  
 14  company_isuzu          205 non-null    uint8  
 15  compan

# Задача 5

Постройте модель с одним предиктором цены – horsepower. Какой процент изменчивости объясняет полученная модель?

In [33]:
#построим модель с одним предиктором цены (price) – horsepower. процент изменчивости =  R-squared
result = smf.ols('price~horsepower', data=df).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Mon, 18 Jul 2022   Prob (F-statistic):           1.48e-48
Time:                        23:19:47   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.0

In [35]:
# полученная модель объяснят 65% процентов изменчивости

# Задача 6

Теперь – две модели:

1. модель со всеми предикторами
2. модель со всеми предикторами, кроме марок машин

Обратите внимание на изменения в R^2, коэффициентах и их значимости. Какую модель лучше оставить? 

In [36]:
#строим две модели
#в первой указываем все предикторы, кроме цены, так как это ЗП
#смотрим на показатель Adjusted 
y = df['price']
df_all_predicts = sm.add_constant(df.drop(columns='price'))
model = sm.OLS(y, df_all_predicts)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Mon, 18 Jul 2022   Prob (F-statistic):           4.86e-89
Time:                        23:23:09   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -3.472e+

In [37]:
#строим две модели
#во второй указываем все предикторы, кроме цены и компаний.
#смотрим на показатель Adjusted и сравниваем с предыдущим шагом
#выбираем ту модель, где меньше предикторов, ведь R2 изменился не очень сильно, а часть марок вообще не значима
y = df['price']
df_not_company_predicts = sm.add_constant(df.drop(columns= ['price', 'company_audi',
                                                            'company_bmw',
                                                            'company_buick',
                                                            'company_chevrolet',
                                                            'company_dodge',
                                                            'company_honda',
                                                            'company_isuzu',
                                                            'company_jaguar',
                                                            'company_mazda',
                                                            'company_mercury',
                                                            'company_mitsubishi',
                                                            'company_nissan',
                                                            'company_peugeot',
                                                            'company_plymouth',
                                                            'company_porsche',
                                                            'company_renault',
                                                            'company_saab',
                                                            'company_subaru',
                                                            'company_toyota',
                                                            'company_volkswagen',
                                                            'company_volvo']))
model = sm.OLS(y, df_not_company_predicts)
results_not_company = model.fit()
print(results_not_company.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Mon, 18 Jul 2022   Prob (F-statistic):           9.86e-81
Time:                        23:23:18   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -1.7e+

Выбранная модель объясняет примерно 
90
 % дисперсии (окр. до целого). Среди предикторов 
10
 из 27 оказались не значимыми (p > 0.05). Пример интерпретации: при единичном изменении показателя horsepower, цена 
ВОЗРАСТАЕТ
 на 
86.8164
 (без округления).